In [81]:
import numpy as np
import pandas as pd

In [108]:
R = pd.read_csv("C:/Users/alsrn/파이썬/프롬데이원 1분기 최근 구매일 점수 .csv")
F = pd.read_csv("C:/Users/alsrn/파이썬/프롬데이원 1분기 주문수 점수 .csv")
M = pd.read_csv("C:/Users/alsrn/파이썬/프롬데이원 1분기 구매액 점수 .csv")
Monetary = pd.read_csv("C:/Users/alsrn/파이썬/프롬데이원 1분기 총 구매액.csv")

In [96]:
from tqdm import tqdm

In [109]:
RF = pd.merge(R, F, on="주문자ID번호")
RFM = pd.merge(RF,M, on="주문자ID번호")
RFM1_add_구매액 = pd.merge(RFM,Monetary,on="주문자ID번호" ) 

In [114]:
rfm_score = RFM1_add_구매액[['주문자ID번호', '총금액', 'R점수','F점수','M점수']]

In [116]:
def get_score(level, data, reverse = False):
    score = [] 
    for j in range(len(data)): 
        for i in range(len(level)): 
            if data[j] <= level[i]: 
                score.append(i+1) 
                break 
            elif data[j] > max(level): 
                score.append(len(level)+1) 
                break 
            else: 
                continue
    if reverse:
        return [len(level)+2-x for x in score]
    else:
        return score 
 
grid_number = 100 ## 눈금 개수, 너무 크게 잡으면 메모리 문제가 발생할 수 있음.
weights = []
for j in range(grid_number+1):
    weights += [(i/grid_number,j/grid_number,(grid_number-i-j)/grid_number)
                  for i in range(grid_number+1-j)]
num_class = 5 ## 클래스 개수
class_level = np.linspace(1,5,num_class+1)[1:-1] ## 클래스를 나누는 지점을 정한다.
total_amount_of_sales = rfm_score['총금액'].sum() ## 구매금액 총합 = 총 매


In [118]:
max_std = 0 ## 표준편차 초기값
for w in tqdm(weights,position=0,desc = '[Finding Optimal weights]'):
    ## 주어진 가중치에 따른 고객별 점수 계산
    score = w[0]*rfm_score['R점수'] + \
                        w[1]*rfm_score['F점수'] + \
                        w[2]*rfm_score['M점수'] 
    rfm_score['Class'] = get_score(class_level,score,True) ## 점수를 이용하여 고객별 등급 부여
    ## 등급별로 구매금액을 집계한다.
    grouped_rfm_score = rfm_score.groupby('Class')['총금액'].sum().reset_index()
        
    ## 클래스별 구매금액을 총구매금액으로 나누어 클래스별 매출 기여도 계산
    grouped_rfm_score['총금액'] = grouped_rfm_score['총금액'].map(lambda x : x/total_amount_of_sales)
    std_sales = grouped_rfm_score['총금액'].std() ## 매출 기여도의 표준편차 계산
    if max_std <= std_sales:
        max_std = std_sales ## 표준편차 최대값 업데이트
        optimal_weights = w  ## 가중치 업데이트

[Finding Optimal weights]:   0%|                                                              | 0/5151 [00:00<?, ?it/s]c:\users\alsrn\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
[Finding Optimal weights]: 100%|███████████████████████████████████████████████████| 5151/5151 [23:43<00:00,  3.62it/s]


In [122]:
print(optimal_weights)

(0.23, 0.02, 0.75)
